In [57]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

#Frame per second to slow down the Video
fps = 15

#Path of the video file
path_to_file = 'solidWhiteRight.mp4'

# Connects to our Video File
cap = cv2.VideoCapture(path_to_file)

# Automatically grab width and height from video feed
# (returns float which we need to convert to integer for later on!)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Writer object which will be used to finally create our outputfile
writer = cv2.VideoWriter('solid_white_right_output.mp4', cv2.VideoWriter_fourcc(*'VIDX'), 20, (width, height))



while cap.isOpened():    
    
    # Read in and grayscale the frame from the video
    _, image = cap.read()
    
    if _ is True:    
        
        # Grab the x and y sizes and make two copies of the image
        # With one copy we'll extract only the pixels that meet our selection,
        # then we'll paint those pixels red in the original image to see our selection 
        # overlaid on the original.

        ysize = image.shape[0]
        xsize = image.shape[1]

        color_select= np.copy(image)
        line_image = np.copy(image)

        # Define our color criteria
        red_threshold, green_threshold, blue_threshold = 200, 200, 200
        rgb_threshold = [red_threshold, green_threshold, blue_threshold]

        # Define a triangular region of interest
        # Keep in mind the origin (x=0, y=0) is in the upper left in image processing

        #Defining the 3 corners of Traingle
#         left_bottom, apex, right_bottom = [155, 730], [700, 400], [1080, 730]
        left_bottom, apex, right_bottom = [-200, 730], [480, 300], [1200, 730]

        #Degining the three lines(line of the form   ax + by + c = 0) made by the three above points
        #                                            |    | 
        #                |------------:)-------------|    |
        #                | |---------:)-------------------|                               
        #                | |
        # Getting the   (a, b) of the left line
        fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)

        #Getting the (a, b) of the right line
        fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)

        #Getting the (a, b) of the bottom line
        fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)

        # Mask pixels below the threshold
        color_thresholds = (image[:,:,0] < rgb_threshold[0]) | \
                            (image[:,:,1] < rgb_threshold[1]) | \
                            (image[:,:,2] < rgb_threshold[2])


        # Find the region inside the lines
        XX, YY = np.meshgrid(np.arange(0, xsize), np.arange(0, ysize))
        region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                            (YY > (XX*fit_right[0] + fit_right[1])) & \
                            (YY < (XX*fit_bottom[0] + fit_bottom[1]))


        # Mask color selection
        color_select[color_thresholds] = [0,0,0]


        # Find where image is both colored right and in the region
        line_image[~color_thresholds & region_thresholds] = [255, 255, 0]


        #Writing our output in 'solid_yellow_left_output.mp4' file
        writer.write(line_image)
        
        # Display our two output images
        cv2.imshow('Lane Detection', line_image)
        
    
        k = cv2.waitKey(1) & 0xFF

        if  k == 13:
            break
        
    else:    
        
        break
        
#Releasing and Destroying all Opened Windows
cap.release()
writer.release()
cv2.destroyAllWindows()